# HiFi-GAN with SageMaker
[HiFi-GAN](https://github.com/jik876/hifi-gan) のトレーニングを SageMaker Training で行います。

## コンテナの作成
* オリジナルの HiFi-GANでは、PyTorch 1.4 を使用しているため、AWS で管理している Deep Learning Container(DLC) の PyTorch 1.4 を使いたいところですが、DLC では不足しているライブラリ(libsndfile1)があるため、DLCベースイメージとしてコンテナをカスタマイズします
  * dlc の詳細は[こちら](https://github.com/aws/deep-learning-containers)
  * dlc で使えるコンテナイメージリストは[こちら](https://github.com/aws/deep-learning-containers/blob/master/available_images.md)

### sm-docker のインストール 
* SageMaker Studio では Docker Image のビルドに sm-docker というツールを使うので、最初にインストールします

In [ ]:
!pip install sagemaker-studio-image-build

### Dockerfile の生成
リージョンの情報とアカウント情報が必要なため取得

In [ ]:
import boto3
region_name = boto3.session.Session().region_name
account_id = boto3.client('sts').get_caller_identity()['Account']

In [ ]:
dockerfile = f"""FROM 763104351884.dkr.ecr.{region_name}.amazonaws.com/pytorch-training:1.4.0-gpu-py3
RUN apt install libsndfile1 -y
"""

with open('./container/Dockerfile','wt') as f:
    f.write(dockerfile)

### Studio アタッチされているロールに以下の信頼関係を追加
以下のコマンドの出力結果をコピーして追加する

In [ ]:
!cat container/trust_relationships.json

### Studio アタッチされているロールにインラインポリシー（もしくはポリシーを別途作成して）アタッチする
以下のコマンドの出力結果をコピーしてアタッチする

In [ ]:
!cat container/inline_policy.json

### コンテナイメージのビルド

In [ ]:
IMAGE_NAME = 'sagemaker-pytorch-hifigan'
TAG = ':1.4'
%cd container
!sm-docker build . --repository {IMAGE_NAME}{TAG}
%cd ../

In [ ]:
image_uri = f'{account_id}.dkr.ecr.{region_name}.amazonaws.com/{IMAGE_NAME}{TAG}'

## トレーニング
### トレーニングデータの取得

In [ ]:
!wget https://data.keithito.com/data/speech/LJSpeech-1.1.tar.bz2 --no-check-certificate
!tar jxvf  LJSpeech-1.1.tar.bz2 > /dev/null 2>&1

### トレーニングデータを S3 にアップロード

In [ ]:
import sagemaker
input_data_uri = sagemaker.session.Session().upload_data(path='LJSpeech-1.1/', bucket=sagemaker.session.Session().default_bucket(), key_prefix='LJSpeech-1.1')

### S3 にアップロードしたファイルを削除
* テスト推論(動作確認)用に 1 ファイルだけ残す

In [ ]:
!cp LJSpeech-1.1/wavs/LJ050-0278.wav ./test_wav
!rm -rf LJSpeech-1.1/wavs/
!rm LJSpeech-1.1.tar.bz2

### トレーニング開始

In [ ]:
from sagemaker.pytorch import PyTorch
estimator = PyTorch(
    image_uri=image_uri,
    entry_point='train.py',
    source_dir = './training_src/',
    instance_count=1,
    instance_type='ml.g4dn.xlarge',
    role=sagemaker.get_execution_role(),
    hyperparameters={
        'config':'config_v1.json',
        'training_epochs':1
    }
)
estimator.fit(input_data_uri)

## モデルの確認
### モデルダウンロード

In [ ]:
job_result = estimator.latest_training_job.describe()

In [ ]:
model_s3_uri = job_result['ModelArtifacts']['S3ModelArtifacts']
!aws s3 cp {model_s3_uri} ./

In [ ]:
!tar zxvf model.tar.gz -C ./model

### 推論に必要なライブラリをインストール

In [ ]:
# !apt update
# !apt install libsndfile1 -y
!wget https://rpmfind.net/linux/opensuse/tumbleweed/repo/oss/x86_64/libsndfile1-1.0.31-3.2.x86_64.rpm
!rpm -ivh libsndfile1-1.0.31-3.2.x86_64.rpm
!pip install librosa

### 推論実行

In [ ]:
%cd inference_src/
!python inference.py --input_wavs_dir ../test_wav --checkpoint_file ../model/g_00000809 --output_dir ../output_wavs_dir
%cd ../